## Evaluation after reducing the dataset from 25000 rows to 10000 rows :

In [97]:
x_reduce = main_df['cleaned_text']
y_reduce = main_df['target']

x_red, _, y_red, _ = train_test_split(x_reduce, y_reduce, test_size=15000, stratify=y_reduce, random_state=42)

reduced_df = pd.DataFrame({'cleaned_text': x_red, 'target': y_red})

print("Number of positive samples:", reduced_df['target'].sum())
print("Number of negative samples:", len(reduced_df) - reduced_df['target'].sum())


Number of positive samples: 5000
Number of negative samples: 5000


In [98]:
reduced_df

,cleaned_text,target
1310,"[assy, mcgee, show, really, certain, age, appr...",1
15793,"[mockney, come, brighton, despite, poor, recep...",0
10349,"[rented, movie, library, hard, find, good, rea...",0
10157,"[movie, definitely, case, style, substance, st...",1
10011,"[ben, stryker, exgreen, beret, stop, little, t...",0
...,...,...
17439,"[favourite, indian, movie, time, comic, genius...",1
6420,"[reason, bad, movie, badly, written, entirely,...",0
10668,"[although, movie, slow, dreamlike, almost, mes...",0
4289,"[unlikely, love, triangle, set, 19th, century,...",1


In [99]:
target_groupby = reduced_df.groupby('target').size()
print(target_groupby)

target
0    5000
1    5000
dtype: int64


In [101]:
a = reduced_df['cleaned_text']
b = reduced_df['target']
display(a, b)

1310     [assy, mcgee, show, really, certain, age, appr...
15793    [mockney, come, brighton, despite, poor, recep...
10349    [rented, movie, library, hard, find, good, rea...
10157    [movie, definitely, case, style, substance, st...
10011    [ben, stryker, exgreen, beret, stop, little, t...
                               ...                        
17439    [favourite, indian, movie, time, comic, genius...
6420     [reason, bad, movie, badly, written, entirely,...
10668    [although, movie, slow, dreamlike, almost, mes...
4289     [unlikely, love, triangle, set, 19th, century,...
10184    [similar, story, line, done, many, time, impro...
Name: cleaned_text, Length: 10000, dtype: object

1310     1
15793    0
10349    0
10157    1
10011    0
        ..
17439    1
6420     0
10668    0
4289     1
10184    0
Name: target, Length: 10000, dtype: int64

In [102]:
a_training, a_testing, b_training, b_testing = train_test_split(a, b, test_size=0.2, stratify=b, random_state=78)

print('Train Test split ratio is : [80, 20]','\n')
print("Training set:")
print("X_train shape:", a_training.shape)
print("y_train shape:", a_testing.shape)


print("\nTesting set:")
print("X_test shape:", b_training.shape)
print("y_test shape:", b_testing.shape)

Train Test split ratio is : [80, 20] 

Training set:
X_train shape: (8000,)
y_train shape: (2000,)

Testing set:
X_test shape: (8000,)
y_test shape: (2000,)


In [103]:
a_training = [' '.join(doc) for doc in a_training]
a_testing = [' '.join(doc) for doc in a_testing]

a_train_tfidf = tfidf_vectorizer.fit_transform(a_training)

a_test_tfidf = tfidf_vectorizer.transform(a_testing)


## Perform GridSearchCV for Random Forest and Gradient Boosting

In [104]:
gscv_rf = GridSearchCV(rf_classifier, param_grid_rf, cv=5, n_jobs=5)

rf_st_time = time.time()
gscv_rf.fit(a_train_tfidf, b_training)
latency_rf = time.time() - rf_st_time

print("Latency for Random Forest: ", latency_rf )

Latency for Random Forest:  701.8885161876678


In [117]:
latency_rf

26.675348154703777

In [115]:
tak_time_rf = latency_rf/60
print(f"Random Forest took {tak_time_rf} minutes to perform the Grid search CV")

Random Forest took 0.4445891359117296 minutes to perform the Grid search CV


# 

In [113]:
bp_rf = gscv_rf.best_params_
bs_rf = gscv_rf.best_score_
print("Best Parameters obtained after performing the GSVC :", bp_rf, '\n')
print("Model Best score :", round(bs_rf, 4)*100)

Best Parameters obtained after performing the GSVC : {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300} 

Model Best score : 85.08


### 

### Going to use the XGB, reason because the Gradient boosting was taking so long to perform GSCV I have almost wait for around 8-10 hours for the result but i did not received the output. So later i decide to move with XGB

### 

## Perform GridSearchCV for XGB :

In [63]:
gsvc_xgb = GridSearchCV(xgb_classifier, param_grid_xgb, cv=5, n_jobs=-5)

start_time_xgb = time.time()

gsvc_xgb.fit(a_train_tfidf, b_training)

latency_xgb = time.time() - start_time_xgb

print("Latency for Random Forest: ", latency_xgb)

Latency for Random Forest:  7599.761258840561


In [66]:
tak_time_xgb = latency_xgb/60
print(f"XGB took {tak_time_xgb} minutes to perform the Grid search CV")

XGB took 126.66268764734268 minutes to perform the Grid search CV


#### Time required to perform GSCV using XGB classifier is 2 hours and around 6 minutes to complete the process.

In [68]:
bp_xgb = gsvc_xgb.best_params_
bs_xgb = gsvc_xgb.best_score_

print("Best Parameters obtained after performing the GSVC :", bp_xgb, '\n')
print("Model Best score :", round(bs_xgb, 4)*100)

Best Parameters obtained after performing the GSVC : {'gamma': 0.2, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 300} 

Model Best score : 85.89


# 

# Perform Final evaluation of models on the best parameter settings using the evaluation metrics

In [84]:
best_model_rf = RandomForestClassifier(**bp_rf)

best_model_rf.fit(a_train_tfidf, b_training)

b_pred_rf = best_model_rf.predict(a_test_tfidf)

accuracy_rf_2 = accuracy_score(b_testing , b_pred_rf)
print("Accuracy of XGB model:", round(accuracy_rf_2, 2)*100)

print("Random Forest Classification Report:")
print(classification_report(b_testing, b_pred_rf))


Accuracy of XGB model: 86.0
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      5000
           1       0.86      0.86      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [85]:
best_model_XGB = xgb.XGBClassifier(**bp_xgb)

best_model_XGB.fit(a_train_tfidf, b_training)

b_pred_xgb = best_model_XGB.predict(a_test_tfidf)

accuracy_xgb = accuracy_score(b_testing, b_pred_xgb)
print("Accuracy of XGB model:", round(accuracy_xgb, 4)*100)

print(" XGB Classification Report:")
print(classification_report(b_testing, b_pred_xgb))




Accuracy of XGB model: 86.59
 XGB Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      5000
           1       0.85      0.88      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

